In [1]:
!pip install --upgrade "coremltools>=7.3.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 5.1 MB/s eta 0:00:00


In [34]:
import torch, coremltools as ct, numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [35]:
# ---------- 1.  Load HF checkpoint -------------------------------------------------
ckpt = "/content/drive/MyDrive/Colab Notebooks/Actionable-Fine-Tune/mobilebert-finetuned-actionable-v2"
tokenizer = AutoTokenizer.from_pretrained(ckpt)
hf_model  = AutoModelForSequenceClassification.from_pretrained(ckpt).eval()

In [36]:
# ---------- 2.  Wrap + invert sigmoid --------------------------------------
class Wrapped(torch.nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model
        self.sig  = torch.nn.Sigmoid()

    def forward(self, input_ids, attention_mask, token_type_ids):
        logit     = self.base(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        ).logits                       # shape [1, 1]
        prob_not  = self.sig(logit)    # P(non-actionable)
        return 1.0 - prob_not          # ← P(actionable)


torch_model = Wrapped(hf_model)

In [37]:
# ---------- 3.  Trace once (makes a TorchScript graph) -----------------------------

L = 8  # dummy length ≥1
example = (
    torch.randint(0, tokenizer.vocab_size, (1, L), dtype=torch.long),
    torch.ones (1, L, dtype=torch.long),
    torch.zeros(1, L, dtype=torch.long)
)
traced = torch.jit.trace(torch_model, example).eval()

/usr/local/lib/python3.11/dist-packages/transformers/models/mobilebert/modeling_mobilebert.py:520: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(1000),


In [38]:
# ---------- 4.  Core ML input spec -------------------------------------------------
max_len = 256
inputs = [
    ct.TensorType("input_ids",      shape=(1, ct.RangeDim(1, max_len)), dtype=np.int32),
    ct.TensorType("attention_mask", shape=(1, ct.RangeDim(1, max_len)), dtype=np.int32),
    ct.TensorType("token_type_ids", shape=(1, ct.RangeDim(1, max_len)), dtype=np.int32),
]


In [42]:
# ---------- 5.  Convert to .mlpackage ---------------------------------------------
mlmodel = ct.convert(
    traced,
    source="pytorch",
    inputs=inputs,
    convert_to="mlprogram",
    compute_precision=ct.precision.FLOAT32,          # use FLOAT16 to shrink size
    package_dir="actionable_classifier.mlpackage"    # <- writes the package directly
)

Running MIL default pipeline:  11%|█▏        | 10/87 [00:00<00:04, 16.22 passes/s]/usr/local/lib/python3.11/dist-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:273: UserWarning: Output, '4158', of the source model, has been renamed to 'var_4158' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 22.65 passes/s]


In [43]:
!zip -r actionable.zip actionable_classifier.mlpackage

  adding: actionable_classifier.mlpackage/ (stored 0%)
  adding: actionable_classifier.mlpackage/Data/ (stored 0%)
  adding: actionable_classifier.mlpackage/Data/com.apple.CoreML/ (stored 0%)
  adding: actionable_classifier.mlpackage/Data/com.apple.CoreML/model.mlmodel (deflated 92%)
  adding: actionable_classifier.mlpackage/Data/com.apple.CoreML/weights/ (stored 0%)
  adding: actionable_classifier.mlpackage/Data/com.apple.CoreML/weights/weight.bin (deflated 7%)
  adding: actionable_classifier.mlpackage/Manifest.json (deflated 60%)


# Try the model - run the script on macOS

In [33]:
import coremltools as ct
from transformers import AutoTokenizer
import numpy as np
import pathlib

# ------------------------------------------------------------
# 1.  Paths
# ------------------------------------------------------------
BASE_DIR = pathlib.Path(__file__).parent                  # folder that holds this script
MLPKG    = BASE_DIR / "actionable_classifier.mlpackage"   # <— your .mlpackage
CKPT     = BASE_DIR / "mobilebert-finetuned-actionable-v2" # tokenizer folder or .tar.gz

# ------------------------------------------------------------
# 2.  Load Core ML model (lazy – graph isn’t compiled yet)
# ------------------------------------------------------------
mlmodel = ct.models.MLModel(str(MLPKG))

# ------------------------------------------------------------
# 3.  Load Hugging Face tokenizer
# ------------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained(str(CKPT))

# ------------------------------------------------------------
# 4.  Helper: sentence → probability
# ------------------------------------------------------------
MAX_LEN = 256      # value you used during convert()

def actionable_prob(text: str) -> float:
    toks = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
    )

    # Core ML wants plain NumPy arrays (dtype=int32 here)
    inputs = {
        "input_ids":      toks["input_ids"].astype(np.int32),
        "attention_mask": toks["attention_mask"].astype(np.int32),
        "token_type_ids": toks["token_type_ids"].astype(np.int32),
    }

    # run on ANE/GPU/CPU (whatever is available)
    out = mlmodel.predict(inputs)

    # Debug: print available keys
    print("Available output keys:", list(out.keys()))

    # Use the first (and likely only) output key
    output_key = list(out.keys())[0]
    return float(out[output_key][0][0])

# ------------------------------------------------------------
# 5.  Demo
# ------------------------------------------------------------
if __name__ == "__main__":
    sentences = [
    # Actionable
    "Please review the quarterly report and send your feedback by Friday.",
    "Fix the login bug in the iOS app as soon as possible.",
    "Schedule a 30-minute meeting with the design team tomorrow at 3 PM.",
    "Approve the budget increase for the marketing campaign today.",
    "Could you deploy the new API version to production tonight?",

    # Not actionable
    "The quarterly report was completed last Friday.",
    "There is a login bug in the current iOS app release.",
    "The design team usually meets on Tuesdays.",
    "Our marketing budget increased by 15 % this quarter.",
    "The new API version went live last night without issues.",
    ]
    for s in sentences:
        p = actionable_prob(s)
        print(f"{p:5.3f}  {s}")


Exception: Model prediction is only supported on macOS version 10.13 or later.